In [ ]:
import torch #main torch module
import torch.nn as nn #neural net module
import torch.optim as optim #optimizers
import torch.nn.functional as F #functions like ReLu Sig Tanh etc
from torch.utils.data import DataLoader #help us with datasets

import torchvision.datasets as datasets #using to access std data
import torchvision.transforms as transforms #transformations

### Network Definitions

This is the older FC neural network
We will now create a CNN 

In [ ]:
class NN(nn.Module):
    def __init__(self, input_size, num_classes): #(28x28)
        #inheritance from nn.Module
        super(NN,self).__init__()
        self.fc1 = nn.Linear(input_size,50)
        self.fc2 = nn.Linear(50,num_classes)
        
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x   

In [ ]:
model = NN(784,10)
x = torch.randn(64,784)
#print(x)
model(x).shape

Below we will define the CNN Model

In [ ]:
class CNN(nn.Module):
    def __init__(self, in_channels, num_classes=10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = in_channels, out_channels=8, 
                               kernel_size=(3,3), stride = (1,1), padding= (1,1))
        self.pool = nn.MaxPool2d(kernel_size = (2,2), stride = (2,2))
        self.conv2 = nn.Conv2d(in_channels = 8, out_channels=16 , 
                               kernel_size=(3,3), stride = (1,1), padding= (1,1))
        self.fc1 = nn.Linear(16*7*7, num_classes)
    
    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        
        return x

In [ ]:
model = CNN(1,10)
x = torch.randn(64, 1, 28, 28)
print(model(x).shape)

### Model Training & Eval.

In [ ]:
# Set Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
device

In [ ]:
# Hyperparameters
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 20

In [ ]:
# Loading Test Data
test_dataset = datasets.MNIST(root='dataset/',
                               train=False, 
                               transform = transforms.ToTensor(),
                               download = True)

test_loader = DataLoader(dataset = test_dataset,
                         batch_size = batch_size,
                         shuffle = True)

In [ ]:
# Loading Data
train_dataset = datasets.MNIST(root='dataset/',
                               train=True, 
                               transform = transforms.ToTensor(),
                               download = True)

train_loader = DataLoader(dataset = train_dataset,
                         batch_size = batch_size,
                         shuffle = True)

In [ ]:
#init the network
model = CNN(in_channels = 1,num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),
                      lr = learning_rate)

In [ ]:
def epoch_accuracy(loader,model):
    #either train or test accuracy
    num_correct = 0
    num_samples = 0
    model.eval() #set to evaluation mode
    with torch.no_grad():
        #only have to check accuracy, dont compute grads
        for x,y in loader:
            x = x.to(device)
            y = y.to(device)
            #x = x.reshape(x.shape[0],-1)
            
            scores = model(x)
            _, predictions = scores.max(1)
            
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
            
    accuracy = float(num_correct)/float(num_samples)
    model.train()
    return accuracy 

In [ ]:
%%time
# training loop
train_accs = []
test_accs = []
for epoch in range(num_epochs):
    print(f"epoch {epoch+1}/{num_epochs} : ",end='\t')
    for batch_idx,(data, targets) in enumerate(train_loader):
        # data to devices
        data = data.to(device)
        targets = targets.to(device)
        #data = data.reshape(data.shape[0],-1)#flattens
        # this is flatten because cnn accepts
        #fwd
        scores = model(data)
        loss = criterion(scores, targets)
        
        #back
        optimizer.zero_grad() #so that it does not store prev backprop calc
        loss.backward()
    
        #gradient desc
        optimizer.step()
    
    train_acc = epoch_accuracy(train_loader, model)
    test_acc = epoch_accuracy(test_loader, model)
    
    train_accs.append(train_acc)
    test_accs.append(test_acc)
    
    print(f"Training Accuracy:{train_acc: .3f}",end='\t')
    print(f"Validation Accuracy:{test_acc: .3f}")

### Training Characteristics

In [ ]:
#!pip install seaborn

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.figure(figsize=(10,5));
sns.lineplot(x = range(num_epochs), y = train_accs, label='Training Accuracy');
sns.lineplot(x = range(num_epochs), y = test_accs, label='Testing Accuracy');